In [156]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary

stopword_more = []
stopword_sastrawi = StopWordRemoverFactory().get_stop_words()

with open("textmining/stoplist.txt", mode="r", encoding="charmap") as stoplist:
    stopword_more = stoplist.read().split()

stopwords = stopword_sastrawi + stopword_more

dictionary = ArrayDictionary(stopwords)
stopwordRemover = StopWordRemover(dictionary)

In [157]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

stemmerFactory = StemmerFactory()
stemmer = stemmerFactory.create_stemmer()

In [158]:
def computeTF(wordDict, bagOfWords):
    tfDict = {}
    bagOfWordsCount = len(bagOfWords)
    for word, count in wordDict.items():
        tfDict[word] = count / float(bagOfWordsCount)
    return tfDict

In [159]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import word_tokenize
import string
import re

def ClearAll(stringData):
    ret = re.sub(r"\d+", "", stringData).translate(str.maketrans("","",string.punctuation)).strip().lower()
    ret = stemmer.stem(ret)
    ret = set(word_tokenize(stopwordRemover.remove(ret)))
    return ret

keywords = []
bow = set([])
data = []

for idx in range(1,51):
    with open("textmining/news_dataset/data{index}.txt".format(index=idx), mode="r", encoding="charmap") as news:
        data.append(news.read())
        keywords.append([])
        keywords[idx-1] = ClearAll(data[idx-1])
        bow = set(bow).union(set(keywords[idx-1]))

In [160]:
import pandas as pd

now = []
score = []

idx=0
for kwd in keywords:
    now.append(dict.fromkeys(bow, 0))
    for word in keywords[idx]:
        now[idx][word] += 1
    scoredf = pd.DataFrame(data=computeTF(now[idx], keywords[idx]), index=[0]).transpose()
    maxtf = float(scoredf.max()/2)
    tmpdf = scoredf[scoredf[0] < maxtf]
    for index,row in tmpdf.iterrows():
        scoredf.drop(index, inplace=True)
    score.append(scoredf.to_dict())
    idx+=1

In [161]:
querylist = "pertumbuhan ekonomi, perkembangan pasar dan pergerakan harga saham"
querylist = ClearAll(querylist)
querylist

{'ekonomi', 'gera', 'harga', 'kembang', 'pasar', 'saham', 'tumbuh'}

In [241]:
rankdocs = []

idx=0
for scr in score:
    totalScore = 0
    for query in querylist:
        if query in score[idx][0]:
            totalScore += score[idx][0][query]
    rankdocs.append({
        "Doc":idx+1,
        "TotalScore":totalScore
    })
    idx+=1

ranked = pd.DataFrame(rankdocs).sort_values("TotalScore", ascending=False).head(10)
ranked

,Doc,TotalScore
47,48,0.040816
1,2,0.030075
41,42,0.029412
42,43,0.026316
7,8,0.025641
38,39,0.024390
48,49,0.024096
5,6,0.023810
4,5,0.023438
8,9,0.022388


In [183]:
from sklearn.preprocessing import LabelEncoder

label = pd.read_csv("textmining/label.csv")
label.head()

,Data,Category
0,data1,ekonomi
1,data2,ekonomi
2,data3,ekonomi
3,data4,ekonomi
4,data5,ekonomi


In [245]:
ranked = pd.concat([label[["Data", "Category"]], ranked["TotalScore"]], axis=1)
ranked = ranked.sort_values("TotalScore", ascending=False).head(10)
ranked["CategoryPredict"] = "ekonomi"
ranked

,Data,Category,TotalScore,CategoryPredict
47,data48,pariwisata,0.040816,ekonomi
1,data2,ekonomi,0.030075,ekonomi
41,data42,pariwisata,0.029412,ekonomi
42,data43,pariwisata,0.026316,ekonomi
7,data8,ekonomi,0.025641,ekonomi
38,data39,ponsel,0.024390,ekonomi
48,data49,pariwisata,0.024096,ekonomi
5,data6,ekonomi,0.023810,ekonomi
4,data5,ekonomi,0.023438,ekonomi
8,data9,ekonomi,0.022388,ekonomi


In [260]:
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

labelencoder = LabelEncoder()

ranked["label_encoder"] = labelencoder.fit_transform(ranked["Category"])
ranked["label_predict_encoder"] = labelencoder.fit_transform(ranked["CategoryPredict"])

recall = []
precission = []
for rg in range(1,11):
    precission.append(precision_score(ranked["label_encoder"].head(rg), ranked["label_predict_encoder"].head(rg), pos_label=0, average="weighted"))
    recall.append(recall_score(ranked["label_encoder"].head(rg), ranked["label_predict_encoder"].head(rg), pos_label=0, average="weighted"))

[0.0,
 0.25,
 0.1111111111111111,
 0.0625,
 0.16,
 0.1111111111111111,
 0.08163265306122448,
 0.140625,
 0.19753086419753085,
 0.25]

In [259]:
zippedRecalPrecission = list(zip(recall, precission))
# recal_precission = pd.DataFrame(zippedList, columns = ['recall', 'precission']).sort_values(by="precission")
recal_precission

,recall,precission
0,0.000000,Data 0.0 Category ...
1,0.500000,Data 0.5 Category ...
2,0.333333,Data 0.333333 Category ...
3,0.250000,Data 0.25 Category ...
4,0.400000,Data 0.4 Category ...
5,0.333333,Data 0.333333 Category ...
6,0.285714,Data 0.285714 Category ...
7,0.375000,Data 0.375 Category ...
8,0.444444,Data 0.444444 Category ...
9,0.500000,Data 0.5 Category ...


In [250]:
import altair as alt

alt.Chart(visual).mark_line().encode(x='recall', y='precission')

alt.Chart(...)

In [238]:
# import matplotlib.pyplot as plt

# plt.plot(precission, recall)
# plt.show